In [16]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced
from sklearn import tree
from sklearn.model_selection import train_test_split
from category_encoders import MEstimateEncoder

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [17]:
credit_df=pd.read_csv('../Resources/credit_record.csv', index_col="ID")
application_df=pd.read_csv('../Resources/application_record.csv', index_col="ID")

In [18]:
credit_df.head()

,MONTHS_BALANCE,STATUS
ID,,
5001711,0,X
5001711,-1,0
5001711,-2,0
5001711,-3,0
5001712,0,C


In [19]:
application_df.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
ID,,,,,,,,,,,,,,,,,
5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [20]:
grouped=credit_df.groupby('ID')['STATUS'].value_counts()
grouped

ID       STATUS
5001711  0          3
         X          1
5001712  0         10
         C          9
5001713  X         22
                   ..
5150483  X         18
5150484  0         12
         C          1
5150485  0          2
5150487  C         30
Name: count, Length: 94143, dtype: int64

In [21]:
credit_grouped=pd.get_dummies(data=credit_df,columns=['STATUS'],
                              prefix='',prefix_sep='').groupby('ID')[sorted(credit_df['STATUS'].unique().tolist())].sum()
credit_grouped=credit_grouped.rename(columns=
                      {'0':'pastdue_1_29',
                       '1':'pastdue_30_59',
                       '2':'pastdue_60_89',
                       '3':'pastdue_90_119',
                       '4':'pastdue_120_149',
                       '5':'pastdue_over_150',
                       'C':'paid_off',
                       'X':'no_loan',
                      })

credit_grouped['number_of_months']=credit_df.groupby('ID')['MONTHS_BALANCE'].count()
credit_grouped

,pastdue_1_29,pastdue_30_59,pastdue_60_89,pastdue_90_119,pastdue_120_149,pastdue_over_150,paid_off,no_loan,number_of_months
ID,,,,,,,,,
5001711,3,0,0,0,0,0,0,1,4
5001712,10,0,0,0,0,0,9,0,19
5001713,0,0,0,0,0,0,0,22,22
5001714,0,0,0,0,0,0,0,15,15
5001715,0,0,0,0,0,0,0,60,60
...,...,...,...,...,...,...,...,...,...
5150482,12,0,0,0,0,0,6,0,18
5150483,0,0,0,0,0,0,0,18,18
5150484,12,0,0,0,0,0,1,0,13


In [22]:
credit_grouped['pastdue_60_89'].value_counts()

pastdue_60_89
0    45388
1      435
2      100
3       39
4       12
5        4
6        3
8        2
7        2
Name: count, dtype: int64

In [23]:
credit_grouped['pastdue_30_59'].value_counts()

pastdue_30_59
0     40784
1      2888
2      1110
3       470
4       268
5       143
6        94
7        63
8        53
9        30
10       23
13       17
11       13
12        8
16        6
15        4
14        2
19        2
18        1
31        1
21        1
27        1
38        1
22        1
17        1
Name: count, dtype: int64

In [24]:
credit_grouped['customer_status'] = credit_grouped['pastdue_30_59'].apply(lambda x: 1 if x > 0 else 0)
credit_grouped

,pastdue_1_29,pastdue_30_59,pastdue_60_89,pastdue_90_119,pastdue_120_149,pastdue_over_150,paid_off,no_loan,number_of_months,customer_status
ID,,,,,,,,,,
5001711,3,0,0,0,0,0,0,1,4,0
5001712,10,0,0,0,0,0,9,0,19,0
5001713,0,0,0,0,0,0,0,22,22,0
5001714,0,0,0,0,0,0,0,15,15,0
5001715,0,0,0,0,0,0,0,60,60,0
...,...,...,...,...,...,...,...,...,...,...
5150482,12,0,0,0,0,0,6,0,18,0
5150483,0,0,0,0,0,0,0,18,18,0
5150484,12,0,0,0,0,0,1,0,13,0


In [25]:
merged_df = pd.merge(credit_grouped[["customer_status"]], application_df, how='inner', left_index=True, right_index=True)

In [26]:
merged_df

,customer_status,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
ID,,,,,,,,,,,,,,,,,,
5008804,1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008805,1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008806,0,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
5008808,0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5008809,0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150482,0,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0
5150483,0,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0
5150484,0,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0


In [ ]:
merged_df['AGE'] = round(-merged_df["DAYS_BIRTH"]/365.25)
merged_df.drop(columns=['DAYS_BIRTH'], inplace=True)

In [ ]:
merged_df['AGE'].unique()

In [ ]:
merged_df['YEARS_EMPLOYED'] = round(-merged_df["DAYS_EMPLOYED"]/365.25)
merged_df.drop(columns=['DAYS_EMPLOYED'], inplace=True)

In [ ]:
merged_df['YEARS_EMPLOYED'].unique()

In [ ]:
#For years employed, negative values mean applicant doesn't work, so we convert negative values to 0 as well
merged_df.loc[(merged_df['YEARS_EMPLOYED'] < 0), 'YEARS_EMPLOYED'] = 0

In [ ]:
for column in merged_df.columns:
    value_counts_result = merged_df[column].value_counts()
    print(f"\nValue counts for column '{column}':\n{value_counts_result}")


In [ ]:
merged_df.isnull().sum()

In [ ]:
merged_df.drop(columns=["FLAG_MOBIL", "FLAG_WORK_PHONE", "FLAG_PHONE", "FLAG_EMAIL"], inplace=True)

In [ ]:
merged_df["YEARS_EMPLOYED"].unique()

In [ ]:
X = merged_df.drop(columns=['customer_status'])
y = merged_df['customer_status']

In [ ]:
from sklearn.impute import SimpleImputer 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(imputer.fit_transform(X_train),index=X_train.index,columns=X_train.columns)

In [ ]:
X_imputed=X_imputed.astype(X_train.dtypes)

In [ ]:
X_imputed.head()

In [ ]:
from category_encoders import MEstimateEncoder
target_encoder=MEstimateEncoder(m=5,cols=['OCCUPATION_TYPE'])
# train the encoder with the 0.25 of the data to prevent overfitting 
X_encode=X_imputed.sample(frac=0.25)
y_encode=y_train[X_encode.index]
target_encoder.fit(X_encode,y_encode)
# transform the categorical feature
X_encoded=target_encoder.transform(X_imputed)
X_encoded['OCCUPATION_TYPE'].unique()

In [ ]:
#encoded_column = 'OCCUPATION_TYPE'

# Get unique encoded values
#encoded_values = X_encoded[encoded_column].unique()

# Access the encoder mapping directly
#mapping_dict = target_encoder.mapping[encoded_column]

# Reverse the dictionary
#reverse_dict = {v: k for k, v in mapping_dict.items()}

# Apply the reverse mapping to get the original occupation types
#original_occupation_types = X_encoded[encoded_column].map(reverse_dict)

In [ ]:
X_encoded

In [ ]:
sns.countplot(x='customer_status', data = merged_df)
plt.show()

In [ ]:
# Separate the data into labels and features

# Separate the y variable, the 
y = merged_df['customer_status']
display(y.head())


In [ ]:
merged_df_dummies = pd.get_dummies(merged_df, columns=['CODE_GENDER','FLAG_OWN_REALTY','FLAG_OWN_CAR','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE'],dtype=int)

merged_df_dummies.head()

In [ ]:
# Separate the X variable, the features
X = merged_df_dummies.drop(columns=['customer_status'])
X.head()

In [ ]:
# Check the balance of our target values
y.value_counts()

In [ ]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Standardize the feature using StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 42 to the models

logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
logistic_regression_model.fit(X_train_scaled,y_train)

In [ ]:
# Make a prediction using the testing data
testing_predictions = logistic_regression_model.predict(X_test_scaled)
print(len(testing_predictions))

In [ ]:
# Print the balanced_accuracy score of the model

logistic_acc_score = balanced_accuracy_score(y_test, testing_predictions)
print(logistic_acc_score)

In [ ]:
# Generate a confusion matrix for the model

confusion_test_matrix = confusion_matrix(y_test, testing_predictions)
confusion_test_matrix

In [ ]:
# Print the classification report for the model
testing_report = classification_report(y_test, testing_predictions)
print(testing_report)

In [ ]:
# Plot confusion matrix heatmap
#plt.figure(figsize=(8,6))
sns.heatmap(confusion_test_matrix,cmap='Reds',annot=True, fmt='d')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confustion Matrix')


In [ ]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res =random_oversampler_model.fit_resample(X_train, y_train)

In [ ]:
# Count the distinct values of the resampled lables data.random_oversampler_model
y_res.value_counts()

In [ ]:
# Standardize the feature using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_res)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
# Instantiate the Logistic Regression model
logistic_regression_model_resampled = LogisticRegression(random_state=1)
logistic_regression_model_resampled

# Fit the model using the resampled training data
logistic_regression_model_resampled.fit(X_train_scaled, y_res)

# Make a prediction using the testing data
y_pred = logistic_regression_model_resampled.predict(X_test_scaled)

In [ ]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Generate a confusion matrix for the model
confusion_matrix_resampled = confusion_matrix(y_test, y_pred)
print(confusion_matrix_resampled)

In [ ]:
classification_report_resampled = classification_report_imbalanced(y_test, y_pred)
print(classification_report_resampled)

In [ ]:
# Plot confusion matrix resampled heatmap

sns.heatmap(confusion_matrix_resampled,cmap='Blues',annot=True, fmt='d')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confustion Matrix Resampled')

In [ ]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [ ]:
# Displaying classification report
print(classification_report(y_test, predictions))

In [ ]:
# Create DOT data
dot_data = tree.export_graphviz(model, out_file=None, feature_names=X_train.columns, class_names=["0","1"], filled= True)

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())